In [2]:
%matplotlib ipympl

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from scipy.special import ker,kei

In [3]:
dstide = xr.open_dataset('data/utide_results.nc')

In [4]:
zi = np.arange(0,len(dstide['height'])-7)
m2i, = np.where(dstide['constituent'] == 'M2')

| Parameter                    | Soulsby (theory) | Codiga (UTide)           |
|------------------------------|------------------|--------------------------|
| major axis                   | $U_a$            | $L^{smaj}_q$             |
| minor axis                   | $U_b$            | $L^{smin}_q$             |
| complex anticlockwise vector | $R_+$            | $a^+_q$                  |
| complex clockwise vector     | $R_-$            | $a^-_q$                  |
| anticlockwise amplitude      | $|R_+|$          | $A^+_q$                  |
| clockwise amplitude          | $|R_-|$          | $A^-_q$                  |
| orientation angle            | $\Phi$           | $\theta_q$               |
| phase lag                    | $\phi$           | $g_q$                    |
| anticlockwise phase          | $\phi_+$         | $\epsilon^+_q$, -$g^+_q$ |
| clockwise phase              | $\phi_-$         | $\epsilon^-_q$, $g^-_q$  |

In [9]:
i = 1j
ap = 0.5*(dstide['Lsmaj']+dstide['Lsmin'])*np.exp(i*(dstide['theta']-dstide['g'])*np.pi/180)
am = 0.5*(dstide['Lsmaj']-dstide['Lsmin'])*np.exp(i*(dstide['theta']+dstide['g'])*np.pi/180)
Ap = np.abs(ap)
Am = np.abs(am)
epsilonp = np.angle(ap)*180/np.pi
epsilonm = np.angle(am)*180/np.pi

Test calculations (makes sure relationships in Equation 9 of Codiga 2011 are satisfied).
All tests should be True.

In [10]:
print(np.sum(~np.isclose(dstide['Lsmaj'],Ap + Am)) == 0)
print(np.sum(~np.isclose(dstide['Lsmin'],Ap - Am)) == 0)
print(np.sum(~(np.isclose(dstide['g'],-epsilonp + dstide['theta']) |
            np.isclose(dstide['g'], -epsilonp + dstide['theta'] + 360))) == 0)

True
True
True


In [11]:
z = np.arange(0.05,150,0.05)
sigma = 1.4e-4
f = 1.19e-4

ustar = 0.0126
zo = 0.0009


def planetary_bbl_structure(ustar,zo,z,f):
    '''
    Calculate vertical structure of a planetary boundary layer (a.k.a.
    bottom Ekman layer) following Soulsby (1983). The calculation assumes
    that the eddy viscosity increases linearly with height.
    '''

    i = 1j 
    kappa = 0.41 # Von Karman's constant
    
    # non-dimensional height and roughness length
    xi = 2*np.sqrt(f*z/(kappa*ustar))
    xio = 2*np.sqrt(f*zo/(kappa*ustar))
    
    # Q/Qinf in Equation 31 of Soulsby (1983)
    fac = (1-(ker(xi)*ker(xio) + kei(xi)*kei(xio))/
             (ker(xio)**2 + kei(xio)**2) + 
           i*(ker(xi)*kei(xio) - kei(xi)*ker(xio))/
             (ker(xio)**2 + kei(xio)**2))

    return fac

### Soulsby test cases

#### Case 1: Rectilinear free stream flow

In [12]:
Rpinf = 0.15
Rminf = 0.15

Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

phip = np.angle(Rp)
phim = -np.angle(Rm)

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

plt.figure()
plt.subplot(121)
plt.plot(np.abs(Rp)/Rpinf,z)
plt.plot(np.abs(Rm)/Rpinf,z)
plt.ylim([0,150])
plt.xlim([0,1.05])
plt.xlabel('normalized amplitude')
plt.ylabel('height [m]')
plt.legend(['AC','CW'])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.subplot(122)
plt.plot(phip*180/np.pi,z)
plt.plot(-phim*180/np.pi,z)
plt.ylim([0,150])
plt.xlim([-0.5,10])
plt.xlabel('phase')
plt.legend(['AC','CW'])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

FigureCanvasNbAgg()

In [13]:
plt.figure()
ax1 = plt.subplot(121)
ax1.plot(Ua,z)
plt.xlim([0,0.3])
plt.ylim([0,150])
plt.xlabel('Ua [m/s]')
plt.ylabel('height [m]')
ax2 = ax1.twiny()
ax2.plot(Ub/Ua,z)
plt.xlim([0,0.6])
plt.ylim([0,150])
plt.xlabel('Ub/Ua')
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)

plt.subplot(122)
plt.plot(phi*180/np.pi,z)
plt.plot(PHI*180/np.pi,z)
plt.ylim([0,150])
plt.legend(['$\phi$','$\Phi$'])
plt.xlabel('$\phi$, $\Phi$ [degrees]')

FigureCanvasNbAgg()

Text(0.5, 0, '$\\phi$, $\\Phi$ [degrees]')

#### Cases 2 and 3: Anticlockwise and clockwise free stream ellipses

In [14]:
Rpinf = 0.75*0.3
Rminf = 0.25*0.3

Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

phip = np.angle(Rp)
phim = -np.angle(Rm)

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

plt.figure()
ax1 = plt.subplot(121)
ax1.plot(Ua,z)
plt.xlim([0,0.3])
plt.ylim([0,150])
plt.xlabel('Ua [m/s]')
plt.ylabel('height [m]')
ax2 = ax1.twiny()
ax2.plot(Ub/Ua,z)
plt.xlim([0,0.6])
plt.ylim([0,150])
plt.xlabel('Ub/Ua')
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)

FigureCanvasNbAgg()

In [17]:
Rpinf = 0.25*0.3
Rminf = 0.75*0.3

Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

phip = np.angle(Rp)
phim = -np.angle(Rm)

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

ax1 = plt.subplot(122)
ax1.plot(Ua,z)
plt.xlim([0,0.3])
plt.ylim([0,150])
plt.xlabel('Ua [m/s]')
ax2 = ax1.twiny()
ax2.plot(Ub/Ua,z)
plt.xlim([0,-0.6])
plt.ylim([0,150])
plt.xlabel('Ub/Ua')
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)